# Optimización de Consultas

creamos la sesión de Spark

In [ ]:
import $ivy.`org.apache.spark::spark-sql:2.4.5` 
import $ivy.`sh.almond::almond-spark:0.6.0`

import org.apache.spark.sql.{NotebookSparkSession, SparkSession}

val spark: SparkSession = 
    NotebookSparkSession
      .builder()
      .appName("Queries Optimization")
      .master("local[*]")
      .getOrCreate()


Logging

In [ ]:
import org.slf4j.LoggerFactory
import org.apache.log4j.{Level, Logger}
Logger.getRootLogger().setLevel(Level.ERROR)

imports

In [ ]:
import spark.implicits._
import spark.sqlContext.implicits._
import org.apache.spark.sql._
import org.apache.spark.sql.{functions => func, _}
import org.apache.spark.sql.types._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types.{IntegerType, StringType, StructType}
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark._
import org.apache.spark.sql.{functions => func, _}
import org.apache.spark.sql.types._, func._

# Los Datos

El dataset ha sido obtenido de:
https://www.ecdc.europa.eu/en/publications-data/download-todays-data-geographic-distribution-covid-19-cases-worldwide

En el se observan los casos diarios de Covid-19 por país hasta el 14-12-20

En la segunda parte se utilizan los datos de las medidas aplicadas a cada país por fecha de inicio y fin:

https://www.ecdc.europa.eu/en/publications-data/download-data-response-measures-covid-19

La última consulta para calcular las infecciones por km2:

https://www.kaggle.com/tanuprabhu/population-by-country-2020

## Creo una clase para trabajar con infecciones 

In [ ]:
case class Infection(day : Int, 
                     month : Int, 
                     year : Int, 
                     nCases: Int, 
                     nDeaths : Int, 
                     country : String,  
                     continent : String) 
extends Serializable

Y un método para medir tiempos de ejecución

In [ ]:
def run[A](code: => A): A = {
    val start = System.currentTimeMillis()
    val res = code
    println(s"Took ${System.currentTimeMillis() - start}")
    res
}

## Calcularemos la media usando solo Scala sin que sea distribuido
(Work in progress)

In [ ]:
  def infectionsScala(lines : List[String]) : List[Infection] =
    lines.map(line => {
      val arr = line.split(",")
      Infection(day = arr(1).toInt,
        month = arr(2).toInt,
        year = arr(3).toInt,
        nCases = arr(4).toInt,
        nDeaths = arr(5).toInt,
        country = arr(6),
        continent = arr(10))
    })

In [ ]:
val content= scala.io.Source.fromFile("data.csv").getLines.map(_.split(";"))

In [ ]:
infections(content)

In [ ]:
  def infectionArrayGrowthAverage(infections : List[Infection]) : Map[String, Int]= {

    val countriesAndCases : Map[String, List[(String, Int)]] = 
      infections.map(x => (x.country,x.nCases))
      .groupBy(_._1)
      
    countriesAndCases.mapValues(x => (x.sum / x.size))
  }

In [ ]:
def average() : = ???

In [ ]:
 val bufferedSource = scala.io.Source.fromFile("data.csv")
  for (line <- bufferedSource.getLines) {
    val cols = line.split(",").map(_.trim)
    // do whatever you want with the columns here
    
      
    println(s"${cols(0)}|${cols(1)}|${cols(2)}|${cols(3)}|${cols(4)}|${cols(5)}|${cols(6)}")
  }
  bufferedSource.close

# Empiezo trabajando con RDDs

In [ ]:
val infectionData = spark.sparkContext.textFile("data.csv")

Creo una funcion para trabajar con un RDD de infecciones

In [ ]:
def infections(lines : RDD[String]) : RDD[Infection] =
    lines.map(line => {
      val arr = line.split(",")
      Infection(
        day = arr(1).toInt,
        month = arr(2).toInt,
        year = arr(3).toInt,
        nCases = arr(4).toInt,
        nDeaths = arr(5).toInt,
        country = arr(6),
        continent = arr(10)
      )
    })

Calculo la media de infecciones diarias por país trabajando con pair RDD

In [ ]:
  def infectionGrowthAverage(infections : RDD[Infection]) : RDD[(String, Int)]= {

    val countriesAndCases : RDD[(String, Iterable[Int])] = 
      infections.map(x => (x.country,x.nCases))
      .groupByKey()
      
    countriesAndCases.mapValues(x => (x.sum / x.size)).sortBy(_._2)
  }

Muestro el resultado y el tiempo de ejecución

In [ ]:
val infectionRDD = infections(infectionData)
val infectionAvgRDD = infectionGrowthAverage(infectionRDD)

In [ ]:
run(
infectionGrowthAverage(infectionRDD)
)
.foreach(println)

# Hago los mismos calculos con un DataFrame

Convierto el RDD obtenido previamente en un DataFrame para inferir la clase infección

In [ ]:
val infectionDF = spark.createDataFrame(infectionRDD)

Utilizo los métodos de la clase DF que incluye uno optimizado para calcular la media.

Ejecuto y comprabamos como el tiempo de ejecución es significativamente menor que en RDD

In [ ]:
run(infectionDF.
    groupBy("country")
    .avg("nCases")
    .orderBy("avg(nCases)")
    .count
)

Otra opción es crear el DataFrame directamente importando los datos pero deja de ser un DF de infecciones

In [ ]:
val dfCovid = spark.read
.option("header", "true")
.option("charset", "UTF8")
.option("delimiter",",")
.option("inferSchema", "true")
.csv("covidworldwide.csv")

In [ ]:
dfCovid.schema

In [ ]:
dfCovid.explain

In [ ]:
run(dfCovid.toDF.groupBy("countriesAndTerritories")
    .agg(mean("cases")).orderBy("avg(cases)")).show(2000)

puedo definir el esquema manualmente para crear el DataFrame

In [ ]:
//Defino el esquema manualmente pero podría verlo importando el csv y viendo como lo hace de base spark

val schema = new StructType()
    .add("dateRep",StringType,true)
    .add("day",IntegerType,true)
    .add("month",IntegerType,true)
    .add("year",IntegerType,true)
    .add("cases",IntegerType,true)
    .add("deaths",IntegerType,true)
    .add("countriesAndTerritories",StringType,true)
    .add("geoId",StringType,true)
    .add("countryterritoryCode",StringType,true)
    .add("popData2018",IntegerType,true)
    .add("continentExp",StringType,true)

In [ ]:
val df = spark.read
.format("csv")
.option("header","true")
.schema(schema)
.load("data.csv")

In [ ]:
df.printSchema()

In [ ]:
df.show()

# Y con un DataSet

In [ ]:
val infectionDS = spark.read
.option("header", "true")
.option("charset", "UTF8")
.option("delimiter",",")
.csv("covidworldwide.csv")
.as[(String,String,String,String,String,String,String,String,String,String,String,String)]

In [ ]:
run(
    infectionDS.groupBy($"countriesAndTerritories")
    .agg(avg($"cases").as[Double])
    .orderBy("avg(cases)")
    .count
    )

### Otras opciones menos eficientes
(Work in progress)

In [ ]:
infectionDS.groupByKey(p => p._7) //hace shuffling de los datos
        .mapValues(p => p._5.toDouble)
        .mapGroups((k,vs) => (???))

In [ ]:
infectionDS.groupByKey(p => p._7) //mas eficiente
        .mapValues(p => p._5.toDouble)
        .reduceGroups((acc,str)=> ???)


In [ ]:
val infGrowAvg = new Aggregator[]{ //utilizando un Aggregator
    
}.toColumn 

### Intento trabajar con DataSet de infecciones Dataset[Infection] pero falla 
(Work in progress)

Esto nos dará error pues no se pueden crear datasets sin tipo de datos y no entiende las infecciones

In [ ]:
val infectionDS = spark.createDataset(infectionRDD).as[(Int, Int, Int, Int, Int, String, String)]

In [ ]:
val infectionDS = spark.createDataset(infectionRDD).as[Infection]

In [ ]:
val ds : Dataset[Infection] = spark.createDataset(infectionRDD).as[Infection]

Necesito importar los Encoders y explicitar el tipo de datos

In [ ]:
import org.apache.spark.sql.Encoders
Encoders.product[Infection]

In [ ]:
import org.apache.spark.sql.{Encoder, Encoders}
import org.apache.spark.sql.Encoders

val dataset = spark.createDataset(infectionRDD)(Encoders.product[Infection])

(A partir de aquí intento crear un DataSet de infecciones pero siempre obtengo el mismo error)

In [ ]:
val infectionDS = spark.createDataset(infectionRDD).as[Infection]

In [ ]:
  def infections(lines : Dataset[(String,String,String,String,String,String,String,String,String,String,String,String)]) 
                       : Dataset[Infection] = 
    lines.map(line => {
      Infection(
        day = line._1.toInt,
        month = line._2.toInt,
        year = line._3.toInt,
        nCases = line._4.toInt,
        nDeaths = line._5.toInt,
        country = line._6,
        continent = line._10
      )
    })

In [ ]:
infections(infectionDS).as[Infection]

# Utilizo una segunda tabla y cruzo datos con RDD, DS y DF

## Creo una consulta para calcular la media de infecciones por Km2

### Utilizando RDDs

In [ ]:
val populationData = spark.sparkContext.textFile("population_by_country_2020.csv")

In [ ]:
case class Population(
    country : String, 
    population : Int, 
    density : Int, 
    land_area: Int, 
    ) 
extends Serializable

Limpio la primera linea del CSV y creo un RDD de población

In [ ]:
val header = populationData.first() 
def population(lines : RDD[String]) : RDD[Population] =
    lines.filter(x => x != header)
    .map(line => {
      val arr = line.split(",")
      Population(
        country = arr(0),
        population = arr(1).toInt,
        density = arr(4).toInt,
        land_area = arr(5).toInt,
      )
    })

Compruebo que se visualizan correctamente los datos

In [ ]:
val populationRDD = population(populationData)
populationRDD.foreach(println)

### Un join computacionalmente pesado desde el principio ya que cruza todos los datos sin quedarnos con los que nos interesen

Spark no me deja hacer un Join de RDD que no sean pair RDD así que tenemos que construirlo

In [ ]:
populationRDD.join(infectionRDD)

Construyo Pair RDDs conservando todos los datos

In [ ]:
val populationByCountry = populationRDD.map(
    x => (x.country,x))

val infectionByCountry = 
      infectionRDD.map(x => (x.country,x))

Hago el Join y agrupo por paises

In [ ]:
val megaRDD = infectionByCountry.join(populationByCountry).groupByKey()

Finalmente calculo la media

In [ ]:
megaRDD.mapValues(
    x => x.map( 
        line => line._1.nCases.toFloat / line._2.land_area.toFloat
    )).mapValues(
    x => x.sum / x.size
).collect()

Lo hago todo en una única operación para calcular el tiempo de ejecución

In [ ]:
run(
    infectionByCountry.join(populationByCountry)
    .groupByKey()
    .mapValues(
    x => x.map( 
        line => line._1.nCases.toFloat / line._2.land_area.toFloat)
    ).mapValues(
        x => x.sum / x.size
    ).collect()
)

¿Hay alguna diferencia cruzando los datos en orden inverso? Parece que no

In [ ]:
run(
    populationByCountry.join(infectionByCountry)
    .groupByKey()
    .mapValues(
    x => x.map( 
        line => line._1.land_area.toFloat / line._2.nCases.toFloat)
    ).mapValues(
        x => x.sum / x.size
    ).collect()
)

#### Para optimizar un poco esta consulta:

Despejo solo los datos que me interesan para trabajar con Pair RDDs y optimizar la consulta

In [ ]:
val countriesAndLandArea = populationRDD.map(
    x => (x.country,x.land_area))

In [ ]:
val countriesAndCases = 
      infectionRDD.map(x => (x.country,x.nCases))
      .groupByKey()

Ejecuto un join y trabajo para calcular primero la media de infecciones por Km2 diaria, 
para luego calcular la media total

In [ ]:
val average = countriesAndCases.join(countriesAndLandArea)

In [ ]:
average.mapValues(
    x => x._1.map(
        y => (y.toFloat / x._2.toFloat)
    )).mapValues(
    x => x.sum/x.size
).collect()

Lo hago todo en una única operación para calcular el tiempo de ejecución

In [ ]:
run(
countriesAndCases.join(countriesAndLandArea)   
.mapValues(
    x => x._1.map(
        y => (y.toFloat / x._2.toFloat)
    )).mapValues(
    x => x.sum / x.size
).collect())

#### Pruebas : producto cartesiano y demás (work in progress)

In [ ]:
populationRDD.cartesian(infectionRDD).take(10)

In [ ]:
val joined = infectionAvgRDD.join(countriesAndLandArea)

In [ ]:
infectionAvgRDD.collect()
countriesAndLandArea.collect()
joined.collect()

In [ ]:
joined.map( x => (x._1,(x._2._1 / x._2._2) : Float)).foreach(println)

### Consulta con DataSet

In [ ]:
val infectionDS = spark.read
.option("header", "true")
.option("charset", "UTF8")
.option("delimiter",",")
.option("inferSchema", "true")
.csv("covidworldwide.csv")
.as[(String,String,String,String,Double,Double,String,String,String,String,String,String)]

In [ ]:
val dsPopulation = spark.read
.option("header", "true")
.option("charset", "UTF8")
.option("delimiter",",")
.option("inferSchema", "true")
.csv("population_by_country_2020.csv")
.as[(String,Float,String,Float,Float,Float,Double,String,String,String,String)]

In [ ]:
run(
infectionDS.join(dsPopulation, $"Country (or dependency)" === $"countriesAndTerritories")
        .select($"Country (or dependency)" as "Country",
                $"dateRep" as "date",
                $"cases",
                $"Land Area (Km\u00b2)",
                $"cases" / $"Land Area (Km\u00b2)" as "infection Per Km\u00b2")
        .groupBy("Country")
        .avg("infection Per Km\u00b2")
        .orderBy(asc("avg(infection Per Km²)"))
        .collect()
    )

### Consulta con DataFrame

In [ ]:
val dfCovid = spark.read
.option("header", "true")
.option("charset", "UTF8")
.option("delimiter",",")
.option("inferSchema", "true")
.csv("covidworldwide.csv")

In [ ]:
val dfMeasures = spark.read
.option("header", "true")
.option("charset", "UTF8")
.option("delimiter",",")
.option("inferSchema", "true")
.csv("response_graphs_data_2021-04-15.csv")
dfMeasures.show
dfMeasures.schema

In [ ]:
val dfPopulation = spark.read
.option("header", "true")
.option("charset", "UTF8")
.option("delimiter",",")
.option("inferSchema", "true")
.csv("population_by_country_2020.csv")
dfPopulation.show
dfPopulation.schema

Modifico los datos de entrada para que el formato fecha se adecue al TimeStamp de Spark

In [ ]:
val dfCovidClean = dfCovid.select($"*",$"dateRep",translate($"dateRep","/","-").as("new-date")).drop("dateRep").show()

Hago una consulta de prueba para obtener la media solo de los casos en España

In [ ]:
val spainCovid = dfCovid.select("dateRep","cases").where("countriesAndTerritories == 'Spain'").toDF

In [ ]:
run(spainCovid.agg(avg("cases"))).show

Cruzo los datos con un Join y hago algunas consultas sencillas

In [ ]:
val megaDF = dfCovid.join(dfMeasures, $"Country" === $"countriesAndTerritories")

In [ ]:
megaDF.select("cases","deaths","dateRep","Response_measure")
    .where("countriesAndTerritories == 'Spain'").show

In [ ]:
run(dfCovid.join(dfMeasures, $"Country" === $"countriesAndTerritories")
        .select("cases","deaths","dateRep","Response_measure")
        .where("countriesAndTerritories == 'Spain'").collect())

### Finalmente ejecuto la consulta de nuestro caso de uso, infecciones por Km2

In [ ]:
run(
dfCovid.join(dfPopulation, $"Country (or dependency)" === $"countriesAndTerritories")
        .select($"Country (or dependency)" as "Country",
                $"dateRep" as "date",
                $"cases",
                $"Land Area (Km\u00b2)",
                $"cases" / $"Land Area (Km\u00b2)" as "infection Per Km\u00b2")
        .groupBy("Country")
        .avg("infection Per Km\u00b2")
        .orderBy(asc("avg(infection Per Km²)"))
        .collect()
    )

# Notas y observaciones personales interesantes 

In [ ]:
dfCovid.select("dateRep","countriesAndTerritories","cases").show //aplico los métodos de DFs

In [ ]:
dfCovid.createOrReplaceTempView("covid") //se pueden aplicar consultas SQL sobre DF

spark.sql("SELECT * FROM covid WHERE countriesAndTerritories == 'Spain'").show

In [ ]:
//filtrados sobre dataframes
dfCovid.filter($"continentExp" === "Asia" || $"continentExp" === "Europe").sort($"continentExp".asc).show()